# 01/20

%matplotlib inline

In [1]:
description = 'train_mobilenet_from_pretrain_v2_tune_valance_arousal_batch_32'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_15:32:41.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
model = tf.keras.models.load_model("./models_h5/01_20_22:12:38_mobilenet_from_pretrain_v2_epoch6_batch_32.h5")

In [8]:
model.trainable = True

In [9]:
want_train = ['feat_arousal', 'feat_valence', 'outputs_valence', 'outputs_arousal']
for n in model.layers:
    if n.name not in want_train:
        n.trainable = False

In [10]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [11]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [12]:
batch_size = 32
epochs = 4

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss # + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 22, "valence_loss": 1.7539684772491455, "arousal_loss": 1.6943268775939941, "emotion_loss": 0.8116551637649536, "time_take": 2.5103800296783447}
{"emotion_correct": 20, "valence_loss": 0.9873614311218262, "arousal_loss": 0.8364459276199341, "emotion_loss": 0.8624258041381836, "time_take": 2.7180533409118652}
{"emotion_correct": 21, "valence_loss": 1.499220609664917, "arousal_loss": 1.3123559951782227, "emotion_loss": 1.0606036186218262, "time_take": 2.8703835010528564}
{"emotion_correct": 25, "valence_loss": 1.1160099506378174, "arousal_loss": 0.950114369392395, "emotion_loss": 0.7665302157402039, "time_take": 3.0850627422332764}
{"emotion_correct": 24, "valence_loss": 0.8861911296844482, "arousal_loss": 0.6699682474136353, "emotion_loss": 0.656857967376709, "time_take": 3.2869625091552734}
{"emotion_correct": 24, "valence_loss": 0.8876493573188782, "arousal_loss": 0.7608251571655273, "emotion_loss": 0.7322604060173035, "time_take": 3.4488983154296875}
{"emotion_cor

{"emotion_correct": 22, "valence_loss": 1.1764295101165771, "arousal_loss": 1.120519757270813, "emotion_loss": 0.9927271604537964, "time_take": 12.237197399139404}
{"emotion_correct": 25, "valence_loss": 0.8973127603530884, "arousal_loss": 0.7201377153396606, "emotion_loss": 0.6989763975143433, "time_take": 12.428892850875854}
{"emotion_correct": 19, "valence_loss": 1.0062406063079834, "arousal_loss": 0.8073654770851135, "emotion_loss": 0.9005831480026245, "time_take": 12.587623834609985}
{"emotion_correct": 19, "valence_loss": 1.0528342723846436, "arousal_loss": 0.9170677661895752, "emotion_loss": 0.8994420170783997, "time_take": 12.787463426589966}
{"emotion_correct": 23, "valence_loss": 0.861380934715271, "arousal_loss": 0.7408355474472046, "emotion_loss": 0.6859170198440552, "time_take": 13.000807523727417}
{"emotion_correct": 19, "valence_loss": 1.2998660802841187, "arousal_loss": 1.082478642463684, "emotion_loss": 1.1253674030303955, "time_take": 13.182730913162231}
{"emotion_cor

{"emotion_correct": 23, "valence_loss": 1.1195471286773682, "arousal_loss": 0.9685310125350952, "emotion_loss": 0.9138703346252441, "time_take": 21.075616598129272}
{"emotion_correct": 23, "valence_loss": 1.1861746311187744, "arousal_loss": 1.078669548034668, "emotion_loss": 0.8430830240249634, "time_take": 21.26012659072876}
{"emotion_correct": 21, "valence_loss": 1.1027873754501343, "arousal_loss": 0.951934814453125, "emotion_loss": 1.03126859664917, "time_take": 21.399864673614502}
{"emotion_correct": 20, "valence_loss": 0.8519412279129028, "arousal_loss": 0.7445626854896545, "emotion_loss": 1.3117996454238892, "time_take": 21.609960556030273}
{"emotion_correct": 19, "valence_loss": 1.010627031326294, "arousal_loss": 0.9462558627128601, "emotion_loss": 0.9275684356689453, "time_take": 21.790491819381714}
{"emotion_correct": 23, "valence_loss": 0.8810668587684631, "arousal_loss": 0.707556962966919, "emotion_loss": 0.6405255794525146, "time_take": 21.999329090118408}
{"emotion_correct

{"emotion_correct": 25, "valence_loss": 1.3493882417678833, "arousal_loss": 1.220069169998169, "emotion_loss": 0.8775332570075989, "time_take": 30.16627311706543}
{"emotion_correct": 25, "valence_loss": 0.7703328132629395, "arousal_loss": 0.5751264095306396, "emotion_loss": 0.6830350756645203, "time_take": 30.358267307281494}
{"emotion_correct": 24, "valence_loss": 1.294429898262024, "arousal_loss": 1.041663646697998, "emotion_loss": 0.544386625289917, "time_take": 30.536280155181885}
{"emotion_correct": 22, "valence_loss": 0.9688500761985779, "arousal_loss": 0.8234019875526428, "emotion_loss": 0.8029906749725342, "time_take": 30.707453966140747}
{"emotion_correct": 27, "valence_loss": 0.847244143486023, "arousal_loss": 0.6366018056869507, "emotion_loss": 0.5573456287384033, "time_take": 30.913920402526855}
{"emotion_correct": 21, "valence_loss": 1.1789608001708984, "arousal_loss": 1.101930856704712, "emotion_loss": 0.7932533621788025, "time_take": 31.04757308959961}
{"emotion_correct"